In [ ]:
from __future__ import print_function
#%matplotlib inline
import random
import torch
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip #unzip data.zip

# Set random seed for reproducibility

In [ ]:
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


# Define some parameters

In [ ]:
# Root directory for dataset
dataroot = './data'

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 32

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 128

# The range of degrees for random rotation
degree = 10.0

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Create the dataset (using ImageFolder)

In [ ]:
# We can use an image folder dataset the way we have it setup.
dataset = dset.ImageFolder(root=dataroot,
              transform=transforms.Compose([
                transforms.Resize(image_size),
                transforms.CenterCrop(image_size),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), 
                # image = (image - mean) / std
                # min: (0 - 0.5) / 0.5 = -1
                # max: (1 - 0.5) / 0.5 = 1
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(degrees=degree, interpolation=Image.BILINEAR, fill=-1.0)
                ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(16,16))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
from collections import Counter
print('Number of Image:')
print(dict(Counter(dataset.targets))[0])

# Weight Initialization

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Generator

In [ ]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 32 x 32
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 64 x 64
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 128 x 128
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

In [ ]:
dummy_input = torch.randn(1, 100, 1, 1).to(device) # batch_size, channels, size, size
out = netG(dummy_input)
out.shape

torch.Size([1, 3, 128, 128])

# Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 128 x 128
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(),
            # state size. (ndf) x 64 x 64
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.LeakyReLU(),
            # state size. (ndf*2) x 32 x 32
            nn.Conv2d(ndf*2, ndf * 2, 4, 2, 1, bias=False),
            nn.LeakyReLU(),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.LeakyReLU(),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.LeakyReLU(),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)

In [ ]:
dummy_input = torch.randn(1, 3, 128, 128).to(device) # batch_size, channels, size, size
out = netD(dummy_input)
out.shape

torch.Size([1, 1, 1, 1])

# Optimizers

In [ ]:
# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_batch = 64
fixed_noise = torch.randn(fixed_batch, nz, 1, 1, device=device)

# Learning rate for optimizers
lr = 1e-4
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.9))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.9))

# Gradient Penalty

In [ ]:
def calc_gradient_penalty(netD, real_data, fake_data, b_size, LAMBDA=10.0):
    alpha = torch.rand(b_size, 1, 1, 1)
    alpha = alpha.to(device)

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    interpolates = interpolates.to(device)
    interpolates = autograd.Variable(interpolates, requires_grad=True)

    disc_interpolates = netD(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, 
                  inputs=interpolates,       
                  grad_outputs=torch.ones(disc_interpolates.size()).to(device),
                  create_graph=True,
                  retain_graph=True,
                  only_inputs=True)[0]
    gradients = gradients.view(gradients.size(0), -1)

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty


# Training

In [ ]:
# Training Loop

# Lists to keep track of progress
G_losses = []
D_losses = []
iters = 0
save_model_path = # The path where the models are saved
save_img_path = # The path where the generated images are saved
base = 0
# Number of training epochs
num_epochs = 500
save_epochs = 50

if base != 0:
  # Load the pre-trained model and continue training
  netD.load_state_dict(torch.load('/Dis_at_ep%d'%base))
  netG.load_state_dict(torch.load('/Gen_at_ep%d'%base))

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        one = torch.FloatTensor([1.0]).to(device)
        mone = (one * -1).to(device)
        ############################
        # (1) Update D network: maximize D(x) - D(G(z)) - Gradient Penalty
        ###########################
        for p in netD.parameters():  # reset requires_grad
          p.requires_grad = True  # they are set to False below in netG update
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = output.mean().view(1)
        # Calculate gradients for D in backward pass
        errD_real.backward(mone)
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = output.mean().view(1)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward(one)
        D_G_z1 = output.mean().item()

        # train with gradient penalty
        gradient_penalty = calc_gradient_penalty(netD, real_cpu, fake, b_size)
        gradient_penalty.backward()

        # Compute error of D as sum over the fake and the real batches and gradient penalty
        errD = errD_fake - errD_real + gradient_penalty
        Wasserstein_D = errD_real - errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize D(G(z))
        ###########################
        for p in netD.parameters():
          p.requires_grad = False  # to avoid computation
        netG.zero_grad()
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = output.mean().view(1)
        # Calculate gradients for G
        errG.backward(mone)
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()
        errG = -errG

        # Output training stats
        if i % 50 == 0 or i == len(dataloader)-1:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tWasserstein_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch+1, num_epochs, i+1, len(dataloader),
                     errD.item(), Wasserstein_D.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            # Save the generated pics
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            fake = fake.detach().numpy()
            plt.figure(figsize=(50, 50))
            for i in range(fixed_batch):
              ax = plt.subplot(8, 8, i + 1)
              b = np.swapaxes(np.swapaxes(fake[i], 0,1),1,2)
              plt.imshow(((b+1)*127.5).astype("uint8"))
              plt.axis("off")

            plt.savefig(save_img_path+"At_ep%d_iter%d.png" %(epoch+base+1, iters+1))
            print("Save pig at ep %d (iter: %d)." %(epoch+base+1, iters+1))

        iters += 1

    if (epoch+1)%save_epochs==0 or epoch==num_epochs-1:
      torch.save(netG.state_dict(), save_model_path+'Gen_at_ep'+str(epoch+base+1))
      torch.save(netD.state_dict(), save_model_path+'Dis_at_ep'+str(epoch+base+1))

      # Plot the training history
      plt.figure(figsize=(10,5))
      plt.title("Generator and Discriminator Loss During Training")
      plt.plot(G_losses,label="G")
      plt.plot(D_losses,label="D")
      plt.xlabel("iterations")
      plt.ylabel("Loss")
      plt.legend()
      plt.savefig(save_img_path+'Loss_ep_'+str(base)+'_to_ep'+str(base+epoch+1)+'.png')